# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,3369157,Cape Town,20.0,190.00,ZA,29131.0,288.25,290.93,292.04,1016.0,77.0,NaN,NaN,-51.968991,-3.848592
1,1,6165406,Thompson,20.0,260.00,CA,24615.0,240.15,246.15,246.15,1023.0,75.0,NaN,NaN,59.750385,-97.258161
2,2,3395981,Maceió,20.0,120.00,BR,29915.0,301.51,299.15,299.15,1013.0,74.0,NaN,NaN,NaN,NaN
3,3,3374210,São Filipe,61.0,56.00,CV,29629.0,295.63,296.29,296.29,1014.0,74.0,1014.0,1009.0,NaN,NaN
4,4,3874787,Punta Arenas,40.0,260.00,CL,2794.0,267.36,278.71,280.15,997.0,65.0,NaN,NaN,-79.100128,-100.457137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,572,2015852,Svetlaya,86.0,297.00,RU,25703.0,248.77,257.03,257.03,1009.0,81.0,1009.0,1008.0,39.219776,44.174836
573,573,3105522,Bereda,98.0,5.81,ES,28036.0,276.55,279.82,281.48,1009.0,99.0,NaN,NaN,70.242092,164.509014
574,574,3449053,São Gotardo,54.0,292.00,BR,29304.0,294.92,293.04,293.04,1012.0,84.0,1012.0,898.0,NaN,NaN
575,575,2352250,Afikpo,90.0,205.00,NG,29826.0,301.48,298.26,298.26,1010.0,85.0,1010.0,998.0,-74.114852,24.920317


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
259,259,1067565,Beloha,91.0,86.0,MG,29682.0,296.78,296.82,296.82,1010.0,71.0,1010.0,993.0,-17.126495,22.856192
475,475,5881688,Fort Saint James,100.0,132.0,CA,2653.0,261.82,265.30,265.30,1019.0,95.0,1019.0,933.0,14.562688,-58.387649
311,311,6167125,Tobermory,99.0,18.0,CA,27215.0,259.46,272.15,272.15,1018.0,74.0,1018.0,994.0,6.183858,24.811060
275,275,3418910,Upernavik,41.0,51.0,GL,25844.0,253.18,258.44,258.44,1028.0,94.0,1028.0,1028.0,65.025204,137.213549
419,419,1641977,Kalianget,100.0,18.0,ID,29962.0,304.14,299.62,299.62,1009.0,80.0,1009.0,1009.0,-27.361994,-62.940129
130,130,2024579,Dzhida,47.0,213.0,RU,25051.0,245.21,250.51,250.51,1038.0,84.0,1038.0,960.0,89.927771,-118.019603
387,387,1078553,Ampanihy,97.0,3.0,MG,2964.0,297.87,296.40,296.40,1010.0,66.0,1010.0,985.0,-0.905073,141.355034
519,519,1497210,Novoagansk,100.0,5.0,RU,25956.0,253.35,259.56,259.56,1018.0,95.0,1018.0,1010.0,25.458067,-27.016907
97,97,1501377,Kungurtug,83.0,165.0,RU,2527.0,248.43,252.70,252.70,1044.0,84.0,1044.0,876.0,-76.409737,-128.630185
44,44,1695736,Panalingaan,100.0,93.0,PH,30002.0,304.16,300.02,300.02,1009.0,74.0,1009.0,1009.0,-46.975717,-149.573964


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City", "Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,259,Beloha,MG,-17.126495,22.856192,
1,475,Fort Saint James,CA,14.562688,-58.387649,
2,311,Tobermory,CA,6.183858,24.811060,
3,275,Upernavik,GL,65.025204,137.213549,
4,419,Kalianget,ID,-27.361994,-62.940129,
5,130,Dzhida,RU,89.927771,-118.019603,
6,387,Ampanihy,MG,-0.905073,141.355034,
7,519,Novoagansk,RU,25.458067,-27.016907,
8,97,Kungurtug,RU,-76.409737,-128.630185,
9,44,Panalingaan,PH,-46.975717,-149.573964,


In [7]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [8]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [9]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = row["Country"]
        print(f'Country = {Country}')
    except:
        print(f'Country = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
        hotels.append(results)
        print(f'Hotel = {results}')
    except:
        results = {"No hotel found nearby"}
        hotels.append(results)
        print(results)
    
    
    try:
        print(f"The closest hotel to {City} is {results}.")
    except (KeyError, IndexError):
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
        
    print("------------")
hotel_df["Hotel Name"] = hotels
hotel_df.rename(columns={"Latitude":"Lat", "Longitude":"Lng"}) #not working???
hotel_df


City = Beloha
Country = MG
Retrieving Results for hotels near Beloha:
{'No hotel found nearby'}
The closest hotel to Beloha is {'No hotel found nearby'}.
------------
City = Fort Saint James
Country = CA
Retrieving Results for hotels near Fort Saint James:
Hotel = The View Hotel
The closest hotel to Fort Saint James is The View Hotel.
------------
City = Tobermory
Country = CA
Retrieving Results for hotels near Tobermory:
Hotel = Escarpment Heights Motel
The closest hotel to Tobermory is Escarpment Heights Motel.
------------
City = Upernavik
Country = GL
Retrieving Results for hotels near Upernavik:
Hotel = Gina's Guesthouse - Upernavik
The closest hotel to Upernavik is Gina's Guesthouse - Upernavik.
------------
City = Kalianget
Country = ID
Retrieving Results for hotels near Kalianget:
Hotel = Family Nur Hotel
The closest hotel to Kalianget is Family Nur Hotel.
------------
City = Dzhida
Country = RU
Retrieving Results for hotels near Dzhida:
{'No hotel found nearby'}
The closest ho

,index,City,Country,Lat,Lng,Hotel Name
0,259,Beloha,MG,-17.126495,22.856192,{No hotel found nearby}
1,475,Fort Saint James,CA,14.562688,-58.387649,The View Hotel
2,311,Tobermory,CA,6.183858,24.811060,Escarpment Heights Motel
3,275,Upernavik,GL,65.025204,137.213549,Gina's Guesthouse - Upernavik
4,419,Kalianget,ID,-27.361994,-62.940129,Family Nur Hotel
5,130,Dzhida,RU,89.927771,-118.019603,{No hotel found nearby}
6,387,Ampanihy,MG,-0.905073,141.355034,{No hotel found nearby}
7,519,Novoagansk,RU,25.458067,-27.016907,{No hotel found nearby}
8,97,Kungurtug,RU,-76.409737,-128.630185,{No hotel found nearby}
9,44,Panalingaan,PH,-46.975717,-149.573964,{No hotel found nearby}


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:

# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…